# Exploring crashes involving pedestrians and cyclists
The IDS worked with the city of Bloomington to obtain data on which crashes involved pedestrians and cyclists. This data can be found in the `data/source-data/bike-crashes-2013-2023.csv` file and the `data/source-data/ped-crashes-2013-2023.csv` file.

In [84]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
from datetime import date, time

pd.set_option('display.max_columns', None)

In [90]:
# read in the bike csv
b_df = pd.read_csv('../data/cleaning-process/bike-crashes-2013-2023.csv')  

In [91]:
# read in the ped csv
p_df = pd.read_csv('../data/cleaning-process/ped-crashes-2013-2023.csv')

In [87]:
# read in a year of data to explore
df_21 = pd.read_csv('../data/clean-data/moco-crash-2022-clean.csv')

In [92]:
df_21

,Agency,Local Code,County,Township,City,Vehicles Involved,Trailers Involved,Number Injured,Number Dead,Number Deer,House Number,Roadway Name,Roadway Suffix,Roadway Number,Roadway Interchange,Roadway Ramp,Roadway Id,Intersecting Road,Intersecting Road Number,Mile Marker,Interchange,Corporate Limits?,Property Type,Feet From,Direction,Latitude,Longitude,Roadway Class,Traffic Control Devices?,Aggressive Driving?,Hit and Run?,Locality,School Zone?,Rumble Strips?,Construction?,Construction Type,Light Condition,Weather Conditions,Surface Condition,Type of Median,Roadway Junction Type,Road Character,Roadway Surface,Primary Factor,Damage Estimate,Manner of Collision,Unique Location Id,State Property Damage?,Traffic Control,DateTime,Address Number,Cyclist Involved,Pedestrian Involved
0,INDIANA UNIV BLOOMINGTON PD,220099,MONROE,BLOOMINGTON,BLOOMINGTON,1,0.0,0,0,0,701,E 17TH,ST,NaN,NaN,NaN,E 17TH,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,39.180909,-86.525593,OTHER,NaN,N,N,URBAN,N,N,N,NaN,DAYLIGHT,CLEAR,DRY,NaN,NO JUNCTION INVOLVED,STRAIGHT/LEVEL,CONCRETE,FAILURE TO YIELD RIGHT OF WAY,UNDER $1001,LEFT TURN,EAST17THST,1,OTHER,2022-01-13 11:08:00,NaN,NaN,True
1,BLOOMINGTON PD,B22-42343,MONROE,BLOOMINGTON,BLOOMINGTON,1,NaN,0,0,0,421,E 3RD ST,NaN,NaN,NaN,NaN,E 3RD ST,NaN,NaN,NaN,NaN,Y,NaN,0.0,NaN,39.164783,-86.528648,PRIVATE DR,NaN,NaN,NaN,URBAN,N,N,N,NaN,DAYLIGHT,CLEAR,DRY,NaN,NO JUNCTION INVOLVED,NaN,ASPHALT,DRIVER DISTRACTED - EXPLAIN IN NARRATIVE,$2501 TO $5000,OTHER - EXPLAIN IN NARRATIVE,E 3RDST,0,NaN,2022-10-13 13:31:00,NaN,NaN,NaN
2,MONROE SD,M22A816,MONROE,RICHLAND,BLOOMINGTON,2,0.0,0,0,0,NaN,NaN,NaN,ST RD 46 W,NaN,NaN,ST RD 46 W,W ARLINGTON,NaN,NaN,NaN,N,NaN,NaN,NaN,39.207648,-86.573792,ST RD,Y,N,N,RURAL,N,N,B,NaN,DAYLIGHT,CLEAR,DRY,NaN,NO JUNCTION INVOLVED,STRAIGHT/LEVEL,ASPHALT,FOLLOWING TOO CLOSELY,$1001 TO $2500,REAR END,ST RD 46 WWARLINGTONRD,0,TRAFFIC CONTROL SIGNAL,2022-01-11 15:19:00,NaN,NaN,NaN
3,BLOOMINGTON PD,B22-02628,MONROE,BLOOMINGTON,BLOOMINGTON,2,NaN,0,0,0,NaN,N DUNN ST,NaN,NaN,NaN,NaN,N DUNN ST,E KIRKWOOD AVE,NaN,NaN,NaN,Y,NaN,0.0,NaN,39.166561,-86.528263,LOCAL/CITY RD,NaN,NaN,NaN,URBAN,N,N,N,NaN,DAYLIGHT,CLOUDY,WET,NaN,FOUR-WAY INTERSECTION,NaN,ASPHALT,FAILURE TO YIELD RIGHT OF WAY,UNDER $1001,RIGHT ANGLE,NDUNNSTEKIRKWOODAVE,0,NaN,2022-01-24 04:27:00,NaN,NaN,NaN
4,ELLETTSVILLE PD,E2200105,MONROE,RICHLAND,ELLETTSVILLE,1,0.0,0,0,1,NaN,NaN,NaN,ST RD 46 W,NaN,NaN,ST RD 46 W,DEER PARK,NaN,NaN,NaN,Y,NaN,100.0,W,39.212153,-86.587526,ST RD,NaN,N,N,URBAN,N,N,N,NaN,DARK (NOT LIGHTED),CLOUDY,DRY,NaN,T-INTERSECTION,CURVE/LEVEL,ASPHALT,ANIMAL/OBJECT IN RDWAY,$2501 TO $5000,COLLISION WITH DEER,DEERPARKDRST RD 46 W,0,LN CONTROL,2022-01-08 08:35:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,MONROE SD,M22A31459,MONROE,WASHINGTON,NaN,2,NaN,0,0,0,NaN,I-69 MM 126.5,NaN,NaN,NaN,NaN,I-69 MM 126.5,NaN,NaN,NaN,NaN,N,NaN,0.0,NaN,39.273519,-86.523203,INTERSTATE,NaN,NaN,NaN,RURAL,N,N,N,NaN,DAYLIGHT,CLEAR,DRY,NaN,NO JUNCTION INVOLVED,NaN,ASPHALT,FAILURE TO YIELD RIGHT OF WAY,$2501 TO $5000,SAME DIRECTION SIDESWIPE,I-69MM126.5,0,NaN,2022-12-28 15:25:00,NaN,NaN,NaN
3642,BLOOMINGTON PD,B22-53872,MONROE,BLOOMINGTON,BLOOMINGTON,1,NaN,1,0,0,NaN,N PETE LLIS DR,NaN,NaN,NaN,NaN,N PETE LLIS DR,E AMY LN,NaN,NaN,NaN,Y,NaN,0.0,NaN,39.168952,-86.495827,LOCAL/CITY RD,NaN,NaN,NaN,URBAN,N,N,N,NaN,DARK (NOT LIGHTED),RAIN,WET,NaN,NO JUNCTION INVOLVED,NaN,ASPHALT,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,$5001 TO $10000,RAN OFF RD,NPETEELLISDREAMYLN,0,NaN,2022-12-30 01:10:00,NaN,NaN,NaN
3643,BLOOMINGTON PD,B22-45386,MONROE,BLOOMINGTON,BLOOMINGTON,2,NaN,2,0,0,NaN,W 20TH ST,NaN,NaN,NaN,NaN,W 20TH ST,W ARLINGTON RD,NaN,NaN,NaN,Y,NaN,0.0,NaN,39.183857,-86.550050,LOCAL/CITY RD,NaN,NaN,NaN,URBAN,N,N,N,NaN,DAYLIGHT,RAIN,WET,NaN,T-INTERSECTION,NaN,ASPHALT,FAILURE TO YIELD RIGHT OF WAY,$5001 TO $10000

Goal: identify matching crashes based on matching variables (date/time, maybe other columns if needed) to add a column which flags bike/ped involvement in each of the crashes in the main data

In [93]:
b_df['Year'] = b_df['DateTime'].apply(get_year)

In [94]:
# b_df_21 = b_df[(b_df['Year'] >= 2013) & (b_df['Year'] <= 2018)]
b_df_21 = b_df[b_df['Year'] == 2022]

In [95]:
merge_df = pd.merge(b_df, df_21, on='DateTime')


In [96]:
merge_p_df = pd.merge(p_df, df_21, on='DateTime')

In [97]:
b_df_21.shape

(38, 52)

In [98]:
merge_df.shape

(38, 104)

In [99]:
b_dates = b_df['DateTime'].reset_index()

In [100]:
b_dates['Bike Flag'] = True

In [101]:
b_dates

,index,DateTime,Bike Flag
0,0,2013-03-23 14:55:00,True
1,1,2013-04-23 16:11:00,True
2,2,2013-01-12 01:24:00,True
3,3,2013-11-06 07:55:00,True
4,4,2013-09-07 16:17:00,True
...,...,...,...
401,401,2022-09-20 09:12:00,True
402,402,2022-11-30 21:32:00,True
403,403,2023-01-20 19:30:00,True
404,404,2017-05-10 20:58:00,True


In [102]:
# df_21.join(bp_dates, on='DateTime')
df_21 = df_21.merge(b_dates, how='left', on='DateTime')


In [103]:
df_21['Bike Flag'].describe()

count       38
unique       1
top       True
freq        38
Name: Bike Flag, dtype: object

In [104]:
p_dates = p_df['DateTime'].reset_index()

In [105]:
p_dates['Ped Flag'] = True

In [106]:
p_dates['DateTime'].apply(get_year).unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2023, 2022])

In [107]:
df_21 = df_21.merge(p_dates, how='left', on='DateTime')

In [108]:
df_21[['Ped Flag','Bike Flag']].describe()

,Ped Flag,Bike Flag
count,58,38
unique,1,1
top,True,True
freq,58,38


In [109]:
#did it work?

master = pd.read_csv('../data/clean-data/master-crashes.csv', low_memory=False)  

In [110]:
bike_ped_crashes = master[(master['Cyclist Involved'] == master['Cyclist Involved']) | (master['Pedestrian Involved'] == master['Pedestrian Involved'])]



In [111]:
# bike_crashes['Year'] = bike_crashes['DateTime'].apply(get_year)
bike_ped_crashes['Primary Factor'].value_counts()

FAILURE TO YIELD RIGHT OF WAY               636
PEDESTRIAN ACTION                           342
OTHER (DRIVER) - EXPLAIN IN NARRATIVE       158
DISREGARD SIGNAL/REG SIGN                    76
UNSAFE BACKING                               67
IMPROPER TURNING                             46
OTHER (DRIVER)                               42
IMPROPER PASSING                             32
DRIVER DISTRACTED - EXPLAIN IN NARRATIVE     23
IMPROPER LN USAGE                            21
FOLLOWING TOO CLOSELY                        19
UNSAFE LN MOVEMENT                           19
UNSAFE SPEED                                 17
DRIVER DISTRACTED                            15
OTHER (ENVIRONMENTAL) - EXPLAIN IN NARR      14
OTHER (VEHICLE) - EXPLAIN IN NARRATIVE       14
WRONG WAY ON ONE WAY                         12
LEFT OF CENTER                               12
VIEW OBSTRUCTED                              10
HEADLIGHT DEFECTIVE OR NOT ON                 9
RAN OFF RD RIGHT                        

In [112]:
crash_df = pd.read_csv('../data/clean-data/master-crashes.csv',low_memory=False)

In [113]:
import altair as alt
# saving data into a file rather than embedding into the chart
alt.data_transformers.enable('json')
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [114]:
def get_year(date):
    return pd.to_datetime(date).year

In [115]:
crash_df['Year'] = crash_df['DateTime'].apply(get_year)

In [116]:
def make_line_chart(source, title, values, x_axis, y_axis):
    return alt.Chart(source, 
          width=500,
          height=300, 
          title=title,
    ).mark_line().encode(
        alt.X(x_axis, axis=alt.Axis(
                values=values,
                grid=True,
                labelAngle=0)
             ),
        alt.Y(y_axis),
    )

In [117]:
# crashes per 1,000 that involved pedestrians or cyclists
counts = []
years = crash_df['Year'].unique()
for year in years:
    year_df = crash_df[crash_df['Year']==year]
    counts.append(year_df[(year_df['Pedestrian Involved'] == year_df['Pedestrian Involved']) | (year_df['Cyclist Involved'] == year_df['Cyclist Involved'])]['Year'].count() / year_df.shape[0] * 1000) 
#     counts.append(year_df[year_df['Pedestrian Involved'] > 0]['Year'].count() / year_df.shape[0])
    
years, counts = zip(*sorted(zip(years, counts)))
source = pd.DataFrame({
  'Year': years,
  'Crashes per 1,000 that Involved Pedestrians or Cyclists': counts
})
title='Crashes per 1,000 that Involved Pedestrians or Cyclists'
values=years
x_axis='Year:O'
y_axis='Crashes per 1,000 that Involved Pedestrians or Cyclists:Q'

make_line_chart(source, title, values, x_axis, y_axis)


alt.Chart(...)

In [118]:
# number of crashes per every 1000 that involved a pedestrian
counts = []
years = crash_df['Year'].unique()
for year in years:
    year_df = crash_df[crash_df['Year']==year]
    counts.append(year_df[year_df['Pedestrian Involved'] == year_df['Pedestrian Involved']]['Year'].count() / year_df.shape[0] * 1000) 
    
years, counts = zip(*sorted(zip(years, counts)))
source = pd.DataFrame({
  'Year': years,
  'Crashes per 1,000 that Involved Pedestrians': counts
})
title='Crashes per 1,000 that Involved Pedestrians'
values=years
x_axis='Year:O'
y_axis='Crashes per 1,000 that Involved Pedestrians:Q'

make_line_chart(source, title, values, x_axis, y_axis)


alt.Chart(...)

In [119]:
# number of crashes per every 1000 that involved a cyclist
counts = []
years = crash_df['Year'].unique()[:-3]
for year in years:
    year_df = crash_df[crash_df['Year']==year]
    counts.append(year_df[year_df['Pedestrian Involved'] == year_df['Pedestrian Involved']]['Year'].count() / year_df.shape[0] * 1000) 
    
years, counts = zip(*sorted(zip(years, counts)))
source = pd.DataFrame({
  'Year': years,
  'Crashes per 1,000 that Involved Pedestrians': counts
})
title='Crashes per 1,000 that Involved Pedestrians'
values=years
x_axis='Year:O'
y_axis='Crashes per 1,000 that Involved Pedestrians:Q'

make_line_chart(source, title, values, x_axis, y_axis)


alt.Chart(...)

In [120]:
def make_bar_chart(source, title, values, x_axis, y_axis,color):
    return alt.Chart(source, 
          width=500,
          height=300, 
          title=title,
    ).mark_bar().encode(
        alt.X(x_axis, axis=alt.Axis(
                values=values,
                grid=True,
                labelAngle=0)
             ),
        alt.Y(y_axis),
        color=color
    )

In [121]:
def severity_flag(death,injury):
    if death > 0:
        return 2
    elif injury > 0:
        return 1
    else:
        return 0

In [122]:
crash_df['Severity'] = crash_df.apply(lambda x: severity_flag(x['Number Dead'], x['Number Injured']), axis=1)